In [ ]:
import os
import re
from pathlib import Path
from collections import Counter

import matplotlib.pyplot as plt
import sklearn

In [ ]:
datadir = Path('~/OneDrive/Documents/data/washer').expanduser()
rawfolder = datadir.joinpath("Raw Data (CoolTerm)")

In [ ]:
os.listdir(rawfolder)

In [ ]:
fnames = {
    'light': '1_Light Cycle Capture (1k to 500Ω) 2022-10-11 10-51-33.txt',
    'lightdryer': '3_Light Cycle+Dryer Capture 2022-10-11 12-40-55.txt',
    'dryer': '2_Dryer Capture 2022-10-11 11-57-47.txt',
}

In [ ]:
raw = {}
for alias, fname in fnames.items():
    with open(rawfolder.joinpath(fname), 'rb') as f:
        raw[alias] = f.read()


In [ ]:
{
    alias: len(data)
    for alias, data in raw.items()
}

In [ ]:
light = raw['light']

In [ ]:
decoded = {
    alias: data.decode('utf-8')
    for alias, data in raw.items()
}

In [ ]:
asbytes = {
    alias: bytearray.fromhex(data)
    for alias, data in decoded.items()
}

In [ ]:
str_ = {
    alias: ''.join(map(chr, data))
    for alias, data in asbytes.items()
}

In [ ]:
len(asbytes['light'])

In [ ]:
asbytes['light'][:20]

In [ ]:
decoded['light'][:60]

In [ ]:
counts = {
    alias: Counter(data)
    for alias, data in asbytes.items()
}

In [ ]:
df_counts = pd.DataFrame.from_dict(counts)

In [ ]:
df_counts.sum(axis=1).sort_values(ascending=False)

In [ ]:
bytes([241])

In [ ]:
b'\x'

In [ ]:
'\xff'

In [ ]:
import sklearn.feature_extraction.text


In [ ]:
import sys
sys.byteorder

In [ ]:
hex(int.from_bytes(bytes.fromhex('e6'), sys.byteorder))[2:]

In [ ]:
bytes.fromhex('e6').hex()

In [ ]:
bytes.hex

In [ ]:
def tokenizer(seq):
    return [hex(b) for b in seq]

In [ ]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    encoding='ascii',
    tokenizer=tokenizer,
    #token_pattern='.',
    ngram_range=(1, 4),
)

In [ ]:
vectorized = vectorizer.fit_transform(asbytes.values())

In [ ]:
vectorized

In [ ]:
pd.Series(vectorized.sum(axis=0).A1).hist()
plt.yscale('log')

In [ ]:
vectorized.sum(axis=0).reshape((-1)).A1.shape

In [ ]:
import binascii

In [ ]:
asbytes['light'][:10]

In [ ]:
binascii.hexlify(asbytes['light'][:10])

In [ ]:
# write bytes to files 
files = []
for alias, data in asbytes.items():
    path = rawfolder.joinpath('{}.raw'.format(alias))
    files.append(path)
    with open(path, 'wb') as f:
        f.write(data)

In [ ]:
# write bytes to files 
files2 = []
for alias, data in str_.items():
    path = rawfolder.joinpath('{}.raw.utf'.format(alias))
    files2.append(path)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(data)

## Tokenizer approach

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer


In [ ]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [ ]:
trainer = BpeTrainer(
    special_tokens=["[UNK]",],
#     min_frequency=3,
    vocab_size=500,
)

In [ ]:
# tokenizer.train([str(rawfolder.joinpath(fname)) for fname in fnames.values()], trainer)

In [ ]:
tokenizer.train([str(path) for path in files2], trainer)

In [ ]:
tokenizer.get_vocab_size()

In [ ]:
tokenizer.get_vocab()

In [ ]:
[[hex(ord(char)) for char in vocab] for vocab in tokenizer.get_vocab()]

In [ ]:
tokenizer.save(str(rawfolder.joinpath('tokenizer.json')))

In [ ]:
xx = tokenizer.encode(str_['light'])

In [ ]:
len(xx)

In [ ]:
xx

In [ ]:
xx.tokens[:20]

In [ ]:
import pandas as pd

In [ ]:
pd.Series(xx.tokens).value_counts()